In [3]:
import sys
import os
import re
import csv
from pathlib import Path
from decimal import Decimal

import importlib
import vcd_parser
import saif_analyzer

importlib.reload(vcd_parser)
importlib.reload(saif_analyzer)

from vcd_parser import VcdParser   # or whatever class/function you actually need
from saif_analyzer import SAIF

In [54]:
# Define file paths and constants -- VCD file and SAIF estimation file
integration_test_dir = "/home/emurphy/Documents/fpga26/eagerly-elastic-container/dynamatic/integration-test"
example = "single_loop"
vcd_file = f"trace.vcd"

# Configuration for switching activity estimation
TCP = 7

# SAIF file
example_name = "single_loop_baseline"
template_folder = "."
result_folder = "."
saif_file = SAIF(template_folder, result_folder, example_name)

In [55]:
# Sensitive list

node_types = set((
    "cmpi",
    "addi",
    "subi",
    "muli",
    "extsi",
    "buffer",
    "load",
    "store",
    "lsq_load",
    "lsq_store",
    "merge",
    "control_merge",
    "fork",
    "d_return",
    "cond_br",
    "end",
    "andi",
    "ori",
    "xori",
    "shli",
    "shrsi"
    "shrui",
    "select",
    "mux",
    "source",
    "sink",
    "trunci",
    "constant",
    "extui"
))

In [ ]:
# Parse the vcd file
vcd_file = f"trace.vcd"
vcd_file = VcdParser(vcd_file)
coefficient = Decimal('1e-9')/vcd_file.timescale_dict["factor"]

# vcd_file.write_saif("vcd_out.saif")
# print([s for s in vcd_file.unique_signal_names if "buffer4" in s])
# print("./vcd_out.saif")

['tb.duv_inst.buffer4_outs_valid', 'tb.duv_inst.buffer4_outs_ready', 'tb.duv_inst.buffer4.clk', 'tb.duv_inst.buffer4.rst', 'tb.duv_inst.buffer4.ins_valid', 'tb.duv_inst.buffer4.ins_ready', 'tb.duv_inst.buffer4.outs_valid', 'tb.duv_inst.buffer4.outs_ready', 'tb.duv_inst.buffer4.fullReg', 'tb.duv_inst.buffer4.outputValid']


In [47]:
###############################################
####### Function Definition
def extract_node_names(signals, node_type_set):
    matched_signals = set()

    # Iterate over each signal
    for signal in signals:
        # Extract the signal names directly under the top wrapper
        wrapper_signal_name = signal.split(".")[-1]
        pattern = re.compile(r'(' + '|'.join(re.escape(nt) for nt in node_types) + r')(\d+)')
        match = re.search(pattern, wrapper_signal_name)

        if match:
            matched_signals.add(match.group(1)+match.group(2))
            
    return sorted(list(matched_signals))      

In [38]:
# Extract complete node list from the vcd file
node_list = extract_node_names(vcd_file.unique_signal_names, node_types)
node_switching = {}

real_clock_cycles = vcd_file.endtime / (coefficient * TCP)

In [43]:
# Update all node switching information
for sel_node in node_list:
    # Update the switching information
    vcd_file.get_node_toggle_count(sel_node)
    
    if sel_node not in node_switching:
        node_switching[sel_node] = []
    
    # Valid Switching
    node_switching[sel_node].append(vcd_file.node_switching_info[sel_node].total_valid_switching)
    
    # Ready Switching
    node_switching[sel_node].append(vcd_file.node_switching_info[sel_node].total_ready_switching)

    # Data Channel Switching
    node_switching[sel_node].append(vcd_file.node_switching_info[sel_node].total_dataout_channel_switching)
    
    # Handshake Channel SA Real
    node_switching[sel_node].append(round(vcd_file.node_switching_info[sel_node].total_handshake_switching / real_clock_cycles, 4))

    # Data Channel SA Real
    node_switching[sel_node].append(round(vcd_file.node_switching_info[sel_node].total_dataout_channel_switching / real_clock_cycles, 4))
with open("switching.txt", "w") as f:
    f.write(str(node_switching))
print("./switching.txt")

./switching.txt


In [32]:
saif_file.write_out_saif()

./single_loop_baseline_pre_all_out.saif
